In [1]:
import csv

# Define the path to the text file
log_txt_file = "0909-ori-20v0.txt"

try:
    # Read the log data from the text file
    with open(log_txt_file, "r") as file:
        lines = file.readlines()

    # Parse the lines and prepare the CSV data
    csv_data = [["SessionTime(seconds)", "EventType", "x", "y", "z", "r"]]  # CSV headers

    for line in lines:
        line = line.strip()
        if line and line[0].isdigit():  # Only process lines with events
            parts = line.split("\t")
            csv_data.append(parts + [""] * (6 - len(parts)))  # Ensure 6 columns in the CSV

    # Write the parsed data to a CSV file
    csv_filename = "log_data.csv"
    with open(csv_filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(csv_data)

    print(f"CSV file saved as {csv_filename}")

except FileNotFoundError:
    print(f"Error: The file {log_txt_file} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


CSV file saved as log_data.csv
